In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas
from pandas import DataFrame
import datetime

In [2]:
def news_crawler(url):
    
    res = requests.get(url)
    
    if res.status_code == 200:
        
        soup = bs(res.text, 'html.parser')
        
        news = soup.select('#main_pack > div > ul > li')

        data = []
        for each in news:
            row = []

            media = each.select('._sp_each_source')

            # 각 기사의 a 태그들에서 헤드라인과 주소를 뽑기 위함
            a_tag = each.findAll('a',{'class' : '_sp_each_title'})
            headline = a_tag[0]['title']
            news_url = a_tag[0]['href']
            summary = each.select('dd')[1]

            row.append(media[0].text.strip() if len(media[0])>0 else "")
            row.append(headline.strip() if len(headline)>0 else "")
            row.append(summary.text.strip() if len(summary)>0 else "")
            row.append(news_url.strip() if len(news_url)>0 else "")

            data.append(row)
    return data

In [3]:
def loadto_end_pages(url,keyword):
    
    now = datetime.datetime.now()
    today = now.strftime('%Y%m%d')  
    
    chk = True
    count = 0
    while chk:
        res = requests.get(url.format(keyword,count,today))
        if res.status_code == 200:
            soup = bs(res.text, 'html.parser')
            
            #결과값 없는 페이지 찾기
            chk_page = soup.select('#notfound')
            if len(chk_page) == 0:
                count += 1
            else:
                chk = False
                break
     
    news_total = []    
    for i in range(0, count+1):
        news_total += news_crawler(url.format(keyword,i,today))
    
    return news_total
        

In [4]:
def save_to_csv(url):
    
    Q_keyword = input('검색할 키워드를 입력해주세요.(해당 키워드 관련 당일 뉴스가 없으면 오류가 납니다.): ')
    keyword = Q_keyword.replace(" ","+")
    
    today_news = loadto_end_pages(url,keyword)
    search_now = datetime.datetime.now()
    search_datetime = search_now.strftime('%Y%m%d %H%M')
    
    df_news_total = DataFrame(today_news)
    df_news_total.columns = ['매체명','헤드라인','내용요약','뉴스url']
    df_news_total.to_csv('./'+search_datetime+'_News_'+keyword+'.csv')
    
    return df_news_total    

In [5]:
#검색어, 날짜 설정(하루단위로), page counting 필요
# sort=2 : 오래된순, sort=1 : 최신순, sort=0 : 관련도 높은순
# 지금은 sort=2 로 고정.

news_url_form = "https://search.naver.com/search.naver?ie=utf8\
&where=news&query={0}&sort=2\
&start={1}1&nso=so:da,p:from{2}to{2},a:all"

save_to_csv(news_url_form)

검색할 키워드를 입력해주세요.(해당 키워드 관련 당일 뉴스가 없으면 오류가 납니다.): 건강식품


,매체명,헤드라인,내용요약,뉴스url
0,넥스트데일리,"풀무원로하스 ‘로젠빈’, 8월31일까지 ‘대한민국 여성건강 독립만세’ 캠페인 진행",풀무원건강생활의 프리미엄 건강기능식품 브랜드 풀무원로하스의 ‘로젠빈’이 광복절을 기...,http://www.nextdaily.co.kr/news/article.html?i...
1,중앙일보,[굿모닝 내셔널]동해 아닌 서해 태안에 오징어 황금어장 떳다,간 해독과 시력 회복에 좋고 성인병을 억제하는 EPA·DHA 등이 함유돼 건강식품으...,http://news.joins.com/article/olink/21426444
2,경북도민일보,"안동단호박, 수출국 日서 인기","이종원 유통특작과장은 ""매년 수출이 증가하고 있는 단호박을 안동의 주력상품으로 키우...",http://www.hidomin.com/news/articleView.html?i...
3,매일신문,벌써 추석 선물 마케팅 나서는 유통가,농`축`수산물과 건강식품 등 총 177개 품목에 대해 최대 75% 할인 판매한다. ...,http://www.imaeil.com/sub_news/sub_news_view.p...
4,경상매일신문,‘안동 단호박’ 일본人 입맛 사로잡아,"이종원 유통특작과장은 ""매년 수출이 증가하고 있는 단호박을 안동의 주력상품으로 키우...",http://www.ksmnews.co.kr/default/index_view_pa...
5,뉴데일리,"이승훈 시장, 청주 농식품 美수출 교두보 마련","이번 수출상담회에서는 전통장류, 청원생명쌀, 한방삼계탕재료, 김치, 건강기능성식품,...",http://www.newdaily.co.kr/news/article.html?no...
6,인천일보,한가로이 한가득 … 미리 준비하는 한가위,"롯데백화점은 다음달 10일까지 정육, 굴비, 청과, 건강식품 선물세트 등 다양한 선...",http://www.incheonilbo.com/?mod=news&act=artic...
7,머니투데이,"우울한 백화점·마트, 폭염 속 '추석 장사' 총력전","배, 사과, 곶감 등 농산 27품목, 한우 등 축산 30품목, 굴비, 갈치 등 수산...",http://www.mt.co.kr/view/mtview.php?type=1&no=...
8,메디컬투데이,"서울시 ‘청년수당 클린카드’, 소주방ㆍ모텔ㆍ대리운전비까지 결제 가능","주류, 건강기능식품 등을 살 수 있는 것이 드러났다. 뿐만 아니라 오토바이 수리점에...",http://www.mdtoday.co.kr/mdtoday/index.html?no...
9,약업신문,‘CPhI Korea 2017’ 8월 22일 코엑스서 개막,특히 올해는 캐나다의 서스캐처원 지역의 건강기능식품 및 원료 산업 현황을 알리기 위...,http://www.yakup.com/news/index.html?mode=view...
